In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
mallards=pd.read_excel("Mallards_initial data.xlsx")
mallards = mallards.loc[:, ~mallards.columns.str.contains('^Unnamed')]
print(mallards.tail(5))

       population  farmed       date       hour  slatitude  slongitude
19910         111       0 2012-10-03   18:46:46  56.440178   13.992034
19911         111       0 2012-10-03   19:46:42  56.439957   13.992407
19912         111       0 2012-10-03   20:47:04  56.440022   13.992071
19913         111       0 2012-10-03   21:49:33  56.440029   13.992332
19914         111       0 2012-10-03   22:48:59  56.440067   13.993050


In [3]:
ordered = mallards.sort_values(by=["population"]).reset_index(drop=True)

In [4]:
ordered.tail(10)

,population,farmed,date,hour,slatitude,slongitude
19905,153,1,2012-10-19,04:39:28,56.500893,13.970902
19906,153,1,2012-10-19,05:39:21,56.500786,13.970841
19907,153,1,2012-10-19,06:38:23,56.500889,13.971055
19908,153,1,2012-10-19,07:38:58,56.487541,13.965097
19909,153,1,2012-10-19,08:39:03,56.487274,13.965044
19910,153,1,2012-10-19,09:39:15,56.487392,13.965101
19911,153,1,2012-10-19,10:38:33,56.487503,13.965265
19912,153,1,2012-10-19,11:38:27,56.487041,13.965001
19913,153,1,2012-10-18,07:41:34,56.487537,13.965165
19914,153,1,2012-10-08,07:59:45,56.432728,13.989956


In [5]:
ordered.to_excel("mallards_ordered.xlsx", index=False)

In [6]:
labelled = pd.DataFrame(columns=["dlatitude", "dlongitude"])

for i, row in ordered.iterrows():
    if i == ordered.index[-1]:
        break
    if row["population"] != ordered.iloc[i+1]["population"]:
        label_row = pd.DataFrame({"dlatitude": row["slatitude"], "dlongitude": row["slongitude"]}, index=[i])
    else:
        label_row = pd.DataFrame({"dlatitude": ordered.iloc[i+1]["slatitude"], "dlongitude": ordered.iloc[i+1]["slongitude"]}, index=[i])
    labelled = pd.concat([labelled, label_row])


In [7]:
labelled.to_excel("mallards_labels.xlsx", index=False)

In [8]:
labelled.head(10)

,dlatitude,dlongitude
0,56.439716,13.991791
1,56.440170,13.992670
2,56.440136,13.992191
3,56.441143,13.988527
4,56.439919,13.991972
5,56.439869,13.992161
6,56.439716,13.992242
7,56.439934,13.991963
8,56.439888,13.992043
9,56.440109,13.991787


In [9]:
# drop last two rows from ordered dataframe
ordered = ordered.iloc[:-2].reset_index(drop=True)

# drop last row from labelled dataframe
labelled = labelled.iloc[:-1].reset_index(drop=True)

In [10]:
print(ordered.iloc[:-1].shape)

(19912, 6)


In [11]:
print(labelled["dlatitude"].iloc[:-1].shape)

(19912,)


In [12]:
X_train, X_test, y_lat_train, y_lat_test = train_test_split(
    ordered.iloc[:-1], labelled["dlatitude"].iloc[:-1], test_size=0.2, random_state=42
)
X_train, X_test, y_long_train, y_long_test = train_test_split(
    ordered.iloc[:-1], labelled["dlongitude"].iloc[:-1], test_size=0.2, random_state=42
)

In [13]:
X_train.shape

(15929, 6)

In [14]:
scaler = StandardScaler()

In [15]:
X_train_no_date = X_train.drop(['date', 'hour'], axis=1)
X_train_scaled = scaler.fit_transform(X_train_no_date)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_no_date.columns, index=X_train_no_date.index)
X_train_scaled_df[['date', 'hour']] = X_train[['date', 'hour']]

In [16]:
X_test_no_date = X_test.drop(['date', 'hour'], axis=1)
X_test_scaled = scaler.transform(X_test_no_date)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_no_date.columns, index=X_test_no_date.index)
X_test_scaled_df[['date', 'hour']] = X_test[['date', 'hour']]

In [17]:
# extract day of the week and hour of the day from datetime columns
X_train_scaled_df['weekday'] = X_train_scaled_df['date'].dt.weekday
X_train_scaled_df['hour'] = pd.to_datetime(X_train_scaled_df['hour'].str.strip(), format='%H:%M:%S')
X_train_scaled_df['hour'] = X_train_scaled_df['hour'].dt.hour

X_test_scaled_df['weekday'] = X_test_scaled_df['date'].dt.weekday
X_test_scaled_df['hour'] = pd.to_datetime(X_test_scaled_df['hour'].str.strip(), format='%H:%M:%S')
X_test_scaled_df['hour'] = X_test_scaled_df['hour'].dt.hour

# drop the original datetime columns
X_train_scaled_df = X_train_scaled_df.drop(['date'], axis=1)
X_test_scaled_df = X_test_scaled_df.drop(['date'], axis=1)

In [18]:
X_train_scaled_df.tail(5)

,population,farmed,slatitude,slongitude,hour,weekday
11284,0.030042,-0.524242,0.125260,0.133639,21,5
11964,0.138355,-0.524242,0.127363,0.126657,19,0
5390,-0.879784,-0.524242,0.125449,0.134608,15,4
860,-1.356359,-0.524242,0.124414,0.135708,18,2
15795,0.939869,-0.524242,0.125983,0.133088,3,0


In [19]:
X_test_scaled_df.tail(5)

,population,farmed,slatitude,slongitude,hour,weekday
16564,1.113169,1.907515,0.123946,0.131636,21,4
19327,1.849695,1.907515,-4.781471,-1.732344,10,0
15022,0.874881,-0.524242,0.126517,0.128668,0,3
2457,-1.139734,-0.524242,0.124135,0.135700,5,1
9210,-0.229908,-0.524242,0.121909,0.135181,17,5


In [20]:
n_units = 32
activation = 'relu'
loss_function = 'mean_squared_error'

In [21]:
model = Sequential()
model.add(Dense(n_units, activation=activation, input_shape=(X_train_scaled_df.shape[1],)))
model.add(Dense(1))

In [22]:
model.compile(optimizer='adam', loss=loss_function)

In [23]:
print(X_train_scaled_df.values.shape)

(15929, 6)


In [24]:
print(y_lat_train.values.shape)

(15929,)


In [25]:
print(model.input_shape)

(None, 6)


In [26]:
X_train_scaled_df.shape

(15929, 6)

In [27]:
X_test_scaled_df.shape

(3983, 6)

In [28]:
historyLat=model.fit(X_train_scaled_df.values, y_lat_train, validation_data=(X_test_scaled_df.values, y_lat_test), epochs=50)

Epoch 1/50
498/498 [==============================] - 1s 3ms/step - loss: 1304.1007 - val_loss: 425.3019
Epoch 2/50
498/498 [==============================] - 1s 2ms/step - loss: 331.6848 - val_loss: 258.7229
Epoch 3/50
498/498 [==============================] - 1s 2ms/step - loss: 211.9365 - val_loss: 156.0933
Epoch 4/50
498/498 [==============================] - 1s 2ms/step - loss: 106.4365 - val_loss: 56.6380
Epoch 5/50
498/498 [==============================] - 1s 2ms/step - loss: 29.7774 - val_loss: 11.2551
Epoch 6/50
498/498 [==============================] - 1s 2ms/step - loss: 5.9099 - val_loss: 2.7435
Epoch 7/50
498/498 [==============================] - 1s 2ms/step - loss: 2.0982 - val_loss: 1.4379
Epoch 8/50
498/498 [==============================] - 1s 2ms/step - loss: 1.2672 - val_loss: 1.0303
Epoch 9/50
498/498 [==============================] - 1s 2ms/step - loss: 0.8816 - val_loss: 0.6973 ETA: 0s - loss: 
Epoch 10/50
498/498 [==============================] - 1s 2ms/ste

In [29]:
historyLong=model.fit(X_train_scaled_df.values, y_long_train, validation_data=(X_test_scaled_df.values, y_long_test), epochs=50)

Epoch 1/50
498/498 [==============================] - 1s 2ms/step - loss: 79.0853 - val_loss: 18.5406
Epoch 2/50
498/498 [==============================] - 1s 2ms/step - loss: 11.3298 - val_loss: 7.1296
Epoch 3/50
498/498 [==============================] - 1s 2ms/step - loss: 5.0998 - val_loss: 3.7759
Epoch 4/50
498/498 [==============================] - ETA: 0s - loss: 2.881 - 1s 2ms/step - loss: 2.8656 - val_loss: 2.1461
Epoch 5/50
498/498 [==============================] - 1s 2ms/step - loss: 1.6050 - val_loss: 1.1510
Epoch 6/50
498/498 [==============================] - 1s 2ms/step - loss: 0.7774 - val_loss: 0.4340
Epoch 7/50
498/498 [==============================] - ETA: 0s - loss: 0.288 - 1s 2ms/step - loss: 0.2824 - val_loss: 0.2083
Epoch 8/50
498/498 [==============================] - 1s 2ms/step - loss: 0.1675 - val_loss: 0.1446
Epoch 9/50
498/498 [==============================] - 1s 2ms/step - loss: 0.1186 - val_loss: 0.1165
Epoch 10/50
498/498 [============================

In [30]:
# Predict the Longitude values for the test set
y_long_pred = model.predict(X_test_scaled_df)

# Compute the mean absolute error and root mean squared error
mae = mean_absolute_error(y_long_test, y_long_pred)
rmse = mean_squared_error(y_long_test, y_long_pred, squared=False)

# Print the MAE and RMSE
print('Mean Absolute Error (MAE):', mae)
print('Root Mean Squared Error (RMSE):', rmse)

Mean Absolute Error (MAE): 0.02481198261442
Root Mean Squared Error (RMSE): 0.11106626480541673
